In [96]:
from dataclasses import dataclass
from typing import List
import json
import requests

@dataclass
class Token:
    symbol: str
    address: str
    decimals: int
        
@dataclass
class Route:
    exchange: str
    amount_out: int
        
USDT = Token("USDT", "0xdac17f958d2ee523a2206206994597c13d831ec7", 6)
DAI = Token("DAI", "0x6B175474E89094C44Da98b954EedeAC495271d0F", 18)
TOKENS = [DAI, USDC]

PARASWAP_API = "https://api.paraswap.io/prices/"

In [17]:
from itertools import permutations

pairs = permutations(TOKENS, 2)

In [70]:
def get_rates(src_token: Token, dest_token: Token, amount: int):
    r = requests.get(PARASWAP_API, params={'srcToken': src_token.address,
                                                    'destToken': dest_token.address,
                                                    'amount': amount,
                                                    'srcDecimals': src_token.decimals,
                                                    'destDecimals': dest_token.decimals,
                                                    'side': 'SELL',
                                                    'network': 1,
                                                    'otherExchangePrices': 'true',
                                                    'partner': 'paraswap.io'},
                                                    headers={
                                                        'sec-fetch-mode': 'navigate',
                                                        'sec-fetch-dest': 'document',
                                                        'sec-fetch-site': 'none',
                                                        'sec-fetch-user': '?1',
                                                        'upgrade-insecure-requests': '1',
                                                        'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36',
                                                        'if-none-match': 'W/"1846-v7KrPAuNYPmQruloB4qmKtJxhlg"',
                                                        'cookie': 'intercom-id-l8fmq3sh=64ca62b4-c939-4ed8-9a71-109e4099c665; intercom-session-l8fmq3sh='
                                                    })
    return json.loads(r.content)

In [90]:
a = get_rates(USDT, DAI, int(300000 * 1e6))
a

https://api.paraswap.io/prices/?srcToken=0xdac17f958d2ee523a2206206994597c13d831ec7&destToken=0x6B175474E89094C44Da98b954EedeAC495271d0F&amount=300000000000&srcDecimals=6&destDecimals=18&side=SELL&network=1&otherExchangePrices=true&partner=paraswap.io


{'priceRoute': {'blockNumber': 14634538,
  'network': 1,
  'srcToken': '0xdac17f958d2ee523a2206206994597c13d831ec7',
  'srcDecimals': 6,
  'srcAmount': '300000000000',
  'destToken': '0x6B175474E89094C44Da98b954EedeAC495271d0F',
  'destDecimals': 18,
  'destAmount': '300044114437576703287346',
  'bestRoute': [{'percent': 100,
    'swaps': [{'srcToken': '0xdac17f958d2ee523a2206206994597c13d831ec7',
      'srcDecimals': 6,
      'destToken': '0x6B175474E89094C44Da98b954EedeAC495271d0F',
      'destDecimals': 18,
      'swapExchanges': [{'exchange': 'ParaSwapPool3',
        'srcAmount': '168000000000',
        'destAmount': '168033059851323525600000',
        'percent': 56,
        'poolAddresses': ['0x7bE351f273Ef11892E4125045D363F56Cb755966',
         '0xA5d07E978398eb1715056D3Ca5Cb31035C02fdAD',
         '0x945BCF562085De2D5875b9E2012ed5Fd5cfaB927'],
        'data': {'version': 4, 'gasUSD': '8.738918'}},
       {'exchange': 'BalancerV2',
        'srcAmount': '132000000000',
        'de

In [99]:
def get_exchanges(swaps):
    exchanges = {}
    for swap in swaps:
        for swap_exchange in swap['swapExchanges']:
            exchanges[swap_exchange['exchange']] = 0
    
    return '+'.join(exchanges.keys())

def get_best_results(data) -> Route:
    exchange = get_exchanges(data['priceRoute']['bestRoute'][0]['swaps'])
    amount_out = int(data['priceRoute']['destAmount'])
    return Route(exchange, amount_out)

def get_other_results(data) -> List[Route]:
    return [Route(res['exchange'], int(res['destAmount'])) for res in data['priceRoute']['others']]
        

In [100]:
get_other_results(a)

[Route(exchange='Saddle', amount_out=92496627095652910781933),
 Route(exchange='ParaSwapPool3', amount_out=300026033405787720000000),
 Route(exchange='ParaSwapPool4', amount_out=299652445115008890000000),
 Route(exchange='ParaSwapPool10', amount_out=298414643273813385300000),
 Route(exchange='Swerve', amount_out=295500495409653532492264),
 Route(exchange='Shell', amount_out=299894999999999999708332),
 Route(exchange='DODOV2', amount_out=300018125053426033200000),
 Route(exchange='UniswapV3', amount_out=299928179852976080188055),
 Route(exchange='OneInchLP', amount_out=921759195937333852434),
 Route(exchange='CurveV2', amount_out=299959958008929094207905),
 Route(exchange='mStable', amount_out=299959504697220011770807),
 Route(exchange='BalancerV2', amount_out=300024645791515637149462),
 Route(exchange='UniswapV2', amount_out=274997722127860528223600),
 Route(exchange='DefiSwap', amount_out=120180138057432181501940),
 Route(exchange='SushiSwap', amount_out=167358002465508982860),
 Route

In [105]:
all_results = [get_best_results(a)] + get_other_results(a)
sorted_results = sorted(all_results, key=lambda d: d.amount_out, reverse=True) 

In [106]:
sorted_results

[Route(exchange='ParaSwapPool3+BalancerV2', amount_out=300044114437576703287346),
 Route(exchange='ParaSwapPool3', amount_out=300026033405787720000000),
 Route(exchange='BalancerV2', amount_out=300024645791515637149462),
 Route(exchange='DODOV2', amount_out=300018125053426033200000),
 Route(exchange='CurveV2', amount_out=299959958008929094207905),
 Route(exchange='mStable', amount_out=299959504697220011770807),
 Route(exchange='UniswapV3', amount_out=299928179852976080188055),
 Route(exchange='Shell', amount_out=299894999999999999708332),
 Route(exchange='ParaSwapPool4', amount_out=299652445115008890000000),
 Route(exchange='ParaSwapPool10', amount_out=298414643273813385300000),
 Route(exchange='Swerve', amount_out=295500495409653532492264),
 Route(exchange='UniswapV2', amount_out=274997722127860528223600),
 Route(exchange='DefiSwap', amount_out=120180138057432181501940),
 Route(exchange='Saddle', amount_out=92496627095652910781933),
 Route(exchange='OneInchLP', amount_out=921759195937